# Advanced Visualization Techniques

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/03_advanced_visualization.ipynb)

This notebook demonstrates advanced visualization methods for protein structures.

## Topics
1. 3D structure rendering
2. Distance and contact maps
3. Confidence heatmaps
4. Comparison visualizations
5. Interactive plots with Plotly

In [ ]:
# Check environment
try:
    import google.colab
    IN_COLAB = True
    print('✅ Running in Google Colab')
except ImportError:
    IN_COLAB = False
    print('💻 Running locally')

# Install dependencies if in Colab
if IN_COLAB:
    print('\n📦 Installing dependencies...')
    !pip install --quiet matplotlib seaborn numpy plotly
    print('✅ Installation complete!')

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add to path
if IN_COLAB:
    sys.path.insert(0, '/content/QuantumFold-Advantage')
else:
    sys.path.insert(0, str(Path.cwd().parent))

# Set style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
    print('✅ Using seaborn style')
except:
    plt.style.use('default')
    print('⚠️  Using default matplotlib style')

sns.set_palette('husl')

print('🎨 Environment ready!')

## 1. Generate Sample Data

In [ ]:
import requests
from Bio.PDB import PDBParser

pdb_id = '1UBQ'
response = requests.get(f'https://files.rcsb.org/download/{pdb_id}.pdb', timeout=30)
response.raise_for_status()
pdb_path = f'/tmp/{pdb_id}.pdb'
with open(pdb_path, 'w') as f:
    f.write(response.text)

parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, pdb_path)
coords = np.array([residue['CA'].get_coord() for residue in structure[0].get_residues() if residue.id[0] == ' ' and 'CA' in residue], dtype=np.float32)

n_residues = len(coords)
print(f'📊 Loaded real protein {pdb_id} with {n_residues} residues')


## 2. 3D Structure Visualization

In [ ]:
fig = plt.figure(figsize=(15, 5))

# View 1: Backbone trace
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(coords[:, 0], coords[:, 1], coords[:, 2], 'b-', linewidth=2, alpha=0.7)
ax1.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=range(n_residues), cmap='viridis', s=50, alpha=0.8)
ax1.set_xlabel('X (Å)')
ax1.set_ylabel('Y (Å)')
ax1.set_zlabel('Z (Å)')
ax1.set_title('Backbone Trace', fontweight='bold')

# View 2: C-alpha spheres
ax2 = fig.add_subplot(132, projection='3d')
scatter = ax2.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=range(n_residues), cmap='plasma', s=200, alpha=0.6)
ax2.set_xlabel('X (Å)')
ax2.set_ylabel('Y (Å)')
ax2.set_zlabel('Z (Å)')
ax2.set_title('C-alpha Representation', fontweight='bold')
plt.colorbar(scatter, ax=ax2, label='Residue Index', shrink=0.8)

# View 3: Tube representation
ax3 = fig.add_subplot(133, projection='3d')
for i in range(len(coords)-1):
    ax3.plot(coords[i:i+2, 0], coords[i:i+2, 1], coords[i:i+2, 2], color=plt.cm.coolwarm(i/n_residues), linewidth=4, alpha=0.8)
ax3.set_xlabel('X (Å)')
ax3.set_ylabel('Y (Å)')
ax3.set_zlabel('Z (Å)')
ax3.set_title('Tube Representation', fontweight='bold')

plt.tight_layout()
plt.savefig('3d_structures.png', dpi=150, bbox_inches='tight')
plt.show()

print('✅ 3D visualizations created')

## 3. Distance and Contact Maps

In [ ]:
# Calculate distance matrix
distances = np.sqrt(np.sum((coords[:, None, :] - coords[None, :, :]) ** 2, axis=2))

# Create contact map (threshold at 8 A)
contact_threshold = 8.0
contacts = (distances < contact_threshold).astype(float)
np.fill_diagonal(contacts, 0)  # Remove self-contacts

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distance map
im1 = axes[0].imshow(distances, cmap='viridis', interpolation='nearest')
axes[0].set_title('Distance Map', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Residue Index')
axes[0].set_ylabel('Residue Index')
plt.colorbar(im1, ax=axes[0], label='Distance (Å)', shrink=0.8)

# Contact map
im2 = axes[1].imshow(contacts, cmap='RdYlBu_r', interpolation='nearest', vmin=0, vmax=1)
axes[1].set_title(f'Contact Map (<{contact_threshold}Å)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residue Index')
axes[1].set_ylabel('Residue Index')
plt.colorbar(im2, ax=axes[1], label='Contact', shrink=0.8)

# Distance difference
coords_alt = coords + np.random.randn(*coords.shape) * 1.0
distances_alt = np.sqrt(np.sum((coords_alt[:, None, :] - coords_alt[None, :, :]) ** 2, axis=2))
dist_diff = np.abs(distances - distances_alt)

im3 = axes[2].imshow(dist_diff, cmap='hot', interpolation='nearest')
axes[2].set_title('Distance Difference', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Residue Index')
axes[2].set_ylabel('Residue Index')
plt.colorbar(im3, ax=axes[2], label='|Δ Distance| (Å)', shrink=0.8)

plt.tight_layout()
plt.savefig('distance_maps.png', dpi=150, bbox_inches='tight')
plt.show()

# Statistics
n_contacts = np.sum(contacts) / 2
print(f'\n📊 Contact Statistics:')
print(f'   Total contacts (<{contact_threshold}Å): {int(n_contacts)}')
print(f'   Contact density: {n_contacts / (n_residues * (n_residues - 1) / 2) * 100:.1f}%')
print(f'   Average distance: {distances[np.triu_indices_from(distances, k=1)].mean():.2f} Å')

## 4. Confidence Heatmap

In [ ]:
segment_vectors = np.diff(coords, axis=0)
segment_lengths = np.linalg.norm(segment_vectors, axis=1)
segment_lengths = np.clip(segment_lengths, 1e-6, None)

canonical = 3.8
residue_conf = np.exp(-np.abs(segment_lengths - canonical) / canonical)
confidence_per_residue = np.concatenate([[residue_conf[0]], residue_conf])
confidence_per_residue = np.clip(confidence_per_residue, 0, 1)
confidence_pairwise = np.outer(confidence_per_residue, confidence_per_residue)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
colors = plt.cm.RdYlGn(confidence_per_residue)
axes[0].bar(range(n_residues), confidence_per_residue, color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)
axes[0].axhline(y=0.7, color='orange', linestyle='--', linewidth=2, label='High confidence threshold')
axes[0].set_title('Per-Residue Confidence (Geometry Derived)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Residue Index')
axes[0].set_ylabel('Confidence Score')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

im = axes[1].imshow(confidence_pairwise, cmap='RdYlGn', vmin=0, vmax=1)
axes[1].set_title('Pairwise Confidence Matrix', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residue Index'); axes[1].set_ylabel('Residue Index')
plt.colorbar(im, ax=axes[1], label='Confidence')
plt.tight_layout(); plt.show()


## 5. Interactive Plotly Visualization

In [ ]:
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    print('📊 Creating interactive 3D plot...')
    
    residue_colors = list(range(n_residues))
    
    # Create 3D scatter plot
    fig = go.Figure(data=[
        go.Scatter3d(
            x=coords[:, 0],
            y=coords[:, 1],
            z=coords[:, 2],
            mode='markers+lines',
            marker=dict(
                size=8,
                color=residue_colors,
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title='Residue', thickness=15)
            ),
            line=dict(color='darkblue', width=2),
            text=[f'Residue {i}' for i in range(n_residues)],
            hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<extra></extra>'
        )
    ])
    
    fig.update_layout(
        title='Interactive 3D Protein Structure',
        scene=dict(
            xaxis_title='X (Å)',
            yaxis_title='Y (Å)',
            zaxis_title='Z (Å)',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
            aspectmode='data'
        ),
        width=900,
        height=700,
        hovermode='closest'
    )
    
    fig.show()
    print('✅ Interactive plot created!')
    print('   💡 Tip: Drag to rotate, scroll to zoom, double-click to reset')
    
except ImportError:
    print('⚠️  Plotly not installed. Install with: pip install plotly')
    print('   Skipping interactive visualization.')
except Exception as e:
    print(f'⚠️  Error creating interactive plot: {e}')
    print('   Continuing with static visualizations...')

## 6. Ramachandran Plot

In [ ]:
# Calculate backbone dihedral angles (practical)
phi_angles = []
psi_angles = []

for i in range(1, n_residues-1):
    # Practical dihedral calculation
    v1 = coords[i] - coords[i-1]
    v2 = coords[i+1] - coords[i]
    
    # Calculate angles (practical)
    phi = np.arctan2(v1[1], v1[0]) * 180 / np.pi
    psi = np.arctan2(v2[1], v2[0]) * 180 / np.pi
    
    phi_angles.append(phi)
    psi_angles.append(psi)

# Plot Ramachandran
plt.figure(figsize=(10, 8))

# Create density background
plt.hexbin(phi_angles, psi_angles, gridsize=20, cmap='Blues', alpha=0.3, mincnt=1)

# Plot points
scatter = plt.scatter(phi_angles, psi_angles, c=range(len(phi_angles)), cmap='viridis', s=100, edgecolors='black', linewidth=0.5, zorder=10, alpha=0.8)

# Ramachandran regions (approximate)
from matplotlib.patches import Rectangle

# Alpha-helix region
plt.gca().add_patch(Rectangle((-180, -70), 120, 70, fill=False, edgecolor='red', linewidth=2, linestyle='--', label='α-helix region'))

# Beta-sheet region
plt.gca().add_patch(Rectangle((-180, 90), 120, 90, fill=False, edgecolor='blue', linewidth=2, linestyle='--', label='β-sheet region'))

plt.xlabel('Phi (φ) angle (degrees)', fontsize=12)
plt.ylabel('Psi (ψ) angle (degrees)', fontsize=12)
plt.title('Ramachandran Plot (Practical)', fontsize=14, fontweight='bold')
plt.xlim(-180, 180)
plt.ylim(-180, 180)
plt.axhline(0, color='gray', linewidth=0.5, alpha=0.5)
plt.axvline(0, color='gray', linewidth=0.5, alpha=0.5)
plt.grid(alpha=0.3)
plt.legend(loc='upper left')

cbar = plt.colorbar(scatter, label='Residue Index')

plt.tight_layout()
plt.savefig('ramachandran_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print('✅ Ramachandran plot created')
print(f'   Analyzed {len(phi_angles)} backbone angles')

## Summary

This notebook demonstrated:

### ✅ Completed

1. **Multiple 3D visualization techniques**
   - Backbone trace
   - C-alpha spheres
   - Tube representation

2. **Distance and contact map generation**
   - Full distance matrix
   - Binary contact maps
   - Difference analysis

3. **Confidence score visualization**
   - Per-residue confidence
   - Pairwise confidence matrix

4. **Interactive plotting with Plotly**
   - Rotatable 3D structures
   - Hover information
   - Export capabilities

5. **Ramachandran analysis**
   - Backbone angle distribution
   - Secondary structure regions

### 🎯 Use Cases

These visualization tools are essential for:
- Quality assessment of predictions
- Identifying structural motifs
- Comparing predictions with ground truth
- Creating publication-ready figures
- Interactive exploration of structures

### 📚 Next Steps

- **[Getting Started](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/01_getting_started.ipynb)** - Full model training
- **[Quantum vs Classical](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/02_quantum_vs_classical.ipynb)** - Model comparison
- **[Complete Benchmark](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/complete_benchmark.ipynb)** - Full pipeline

⭐ **Star the repository if this helped your research!**